# Dask on HPC Clusters

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dask-on-HPC-Clusters" data-toc-modified-id="Dask-on-HPC-Clusters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dask on HPC Clusters</a></span><ul class="toc-item"><li><span><a href="#Learning-Objectives" data-toc-modified-id="Learning-Objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Learning Objectives</a></span></li><li><span><a href="#Dask-Jobqueue" data-toc-modified-id="Dask-Jobqueue-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dask Jobqueue</a></span><ul class="toc-item"><li><span><a href="#How-this-works" data-toc-modified-id="How-this-works-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>How this works</a></span><ul class="toc-item"><li><span><a href="#DAV-(Casper)---SLURM" data-toc-modified-id="DAV-(Casper)---SLURM-1.2.1.1"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>DAV (Casper) - SLURM</a></span></li><li><span><a href="#Cheyenne-or-Hobart---PBS" data-toc-modified-id="Cheyenne-or-Hobart---PBS-1.2.1.2"><span class="toc-item-num">1.2.1.2&nbsp;&nbsp;</span>Cheyenne or Hobart - PBS</a></span></li></ul></li></ul></li><li><span><a href="#NCAR-Jobqueue" data-toc-modified-id="NCAR-Jobqueue-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>NCAR Jobqueue</a></span><ul class="toc-item"><li><span><a href="#Create-and-Connect-to-Dask-Distributed-Cluster" data-toc-modified-id="Create-and-Connect-to-Dask-Distributed-Cluster-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Create and Connect to Dask Distributed Cluster</a></span></li><li><span><a href="#Scale-our-cluster" data-toc-modified-id="Scale-our-cluster-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Scale our cluster</a></span></li><li><span><a href="#Adaptive-Scaling" data-toc-modified-id="Adaptive-Scaling-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>Adaptive Scaling</a></span></li></ul></li><li><span><a href="#Debugging" data-toc-modified-id="Debugging-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Debugging</a></span></li><li><span><a href="#Monitoring-your-jobs" data-toc-modified-id="Monitoring-your-jobs-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Monitoring your jobs</a></span></li><li><span><a href="#Close/Destroy-Cluster" data-toc-modified-id="Close/Destroy-Cluster-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Close/Destroy Cluster</a></span></li><li><span><a href="#Going-Further" data-toc-modified-id="Going-Further-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Going Further</a></span></li></ul></li></ul></div>

## Learning Objectives

- Learn how to deploy dask on multiple HPC machines using PBS, SLURM, etc via `dask-jobqueue` package .....
- Learn how to take advantage of `ncar-jobqueue` package when using `dask-jobqueue` on NCAR machines
- Learn how to take advantage of dask's **adaptive scaling**. Dask offers the ability to “autoscale” clusters based on a set of heuristics. When the cluster needs more CPU or memory, it will scale up. When the cluster has unused resources, it will scale down.

## Dask Jobqueue

In the notebook on [Dask-distributed scheduler](03_distributed.ipynb), we learned how to create and connect to a local distributed cluster. 

In [ ]:
from dask.distributed import Client, LocalCluster
# Setup a local cluster.
# By default this sets up 1 worker per core
local_cluster = LocalCluster()
local_client = Client(local_cluster)
local_cluster

This code cell above works on both your personal laptop and any HPC cluster (on the HPC machine such as Cheyenne, Casper, the dask workers will be created on a single node). It's when you want to run `dask-workers` on multiple machines (nodes) on an HPC system that `dask-jobqueue` comes in handy. 

[`Dask-jobqueue`](https://jobqueue.dask.org/en/latest/) allows us to seamlessly deploy dask on HPC clusters that use a variety of job queuing systems such as PBS, Slurm.

### How this works


In short, to set up a Dask scheduler and be ready to start jobs providing dask workers, you need to run something like:


#### DAV (Casper) - SLURM

```python
from dask_jobqueue import SLURMCluster
from dask.distributed import Client

cluster = SLURMCluster(processes=4,
                       threads=2,
                       memory="16GB",
                       project="project_id",
                       walltime="01:00:00",
                       queue="queue_name")
client = Client(cluster)
```

#### Cheyenne or Hobart - PBS

```python
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(processes=4,
                       threads=2,
                       memory="16GB",
                       project="project_id",
                       walltime="01:00:00",
                       queue="queue_name")
client = Client(cluster)
```


- This will create a Dask scheduler and wait for you to scale the cluster up using, e.g.,

```pyhton
cluster.scale(8)
```


- This will lead to the submission of a sufficient number of SLURM or PBS jobs to provide 8 dask workers.

## NCAR Jobqueue

When using `dask-jobqueue`, the user has to use the appropriate queueing system class i.e PBSCluster or SLURMCluster depending on which machine they are running on. The implication of this behavior is that everytime you switch from one machine (e.g., Cheyenne) to another (e.g., Casper), you will need to replace `PBSCluster()` call with `SLURMCluster()`call and vice-versa. To improve interoperability between NCAR machines, we provide [`ncar-jobqueue`](https://github.com/NCAR/ncar-jobqueue) package, which is a thin wrapper around `dask-jobqueue` with utilities for expanding `dask-jobqueue` with appropriate settings for NCAR's clusters namely **Cheyenne**, **Casper (DAV)**, **CGD's Hobart**.

### Create and Connect to Dask Distributed Cluster

In [ ]:
from ncar_jobqueue import NCARCluster
from dask.distributed import Client

In [ ]:
cluster = NCARCluster(project="project_id", queue="queue_name") # you can specify more options i.e. memory, etc..
cluster

☝️ Don't forget to click the link above to view the scheduler dashboard! (you may wish to have both the notebook and dashboard side-by-side)

In [ ]:
client = Client(cluster) # Connect to a Dask cluster in order to submit computation
client

### Scale our cluster

In [ ]:
cluster.scale(2) 

- This will lead to the submission of a sufficient number of SLURM or PBS jobs to provide 2 dask workers.

### Adaptive Scaling

Unlike `MPI`, `dask` can scale adaptively, that is, you can give dask a configuration of minimum workers to run continously, and the maximum of workers to scale up to when more workers are needed. Dask will automatically figure out when to ask for more workers (submit more jobs to the queue). When dask determines that it does not need the extra workers (due to reduced workload), it will release them (kill them), and get them back when needed. 


So, instead of scaling our dask to a fixed number of workers with `cluster.scale(2)`,  Dask-jobqueue supports **auto-scaling** the dask cluster with a simple interface:

In [ ]:
cluster.adapt(minimum=3, maximum=5)

In this scenario, dask will maintain at least 3 dask workers running all the time. When the cluster needs more CPU or memory, it will scale up to 5 dask workers. 
When the cluster has unused resources, it will scale down to the minimum of 3 workers.

## Debugging


One thing that might help you initially is the ability to see the job script that will be submitted to the job scheduler:

```python
print(cluster.job_script())
```

A typical job script will look similar to this:

```bash
#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p dav
#SBATCH -n 1
#SBATCH --cpus-per-task=24
#SBATCH --mem=94G
#SBATCH -t 00:30:00
JOB_ID=${SLURM_JOB_ID%;*}

/path/to/python -m distributed.cli.dask_worker tcp://xx.xx.xx.xx:xxxxx \
    --nthreads 12 --nprocs 2 --memory-limit 50.00GB --name dask-worker--${JOB_ID}-- \
    --death-timeout 15s --local-directory /tmp --interface ib0
```

In [ ]:
# Let's take a look at our job script
print(cluster.job_script())

## Monitoring your jobs


- To see what your dask-Jobqueue cluster is doing to the HPC job scheduler, a few shell commands might come in handy:

See all you jobs with:

```bash
squeue -u $USER
```

or 

```bash
qstat -u $USER
```



- or wrap it in a watch (but make sure to be patient and set a reasonable/appropriate interval with the `-n` flag:



```bash
watch -n 60 squeue -u  $USER
```

or 

```bash
watch -n 60 qstat -u  $USER
```




- To get a quick overview of the number of pending (PD), running (R), etc. jobs with:


```bash
watch -n 60 "squeue | grep $USER | awk '{print \$5}' | sort | uniq -c | paste -s"
```

or


```bash
watch -n 60 "qstat | grep $USER | awk '{print \$5}' | sort | uniq -c | paste -s"
```


- To monitor all your processes, try:

```bash
top -u ${USER}
```

or

```bash
htop -u ${USER}
```

## Close/Destroy Cluster

To close our cluster and client when you are done with your work, run:

In [ ]:
client.close()

In [ ]:
cluster.close()

Make sure that your jobs submitted via `dask-jobqueue` were terminated or being terminated:

In [ ]:
# !squeue -u $USER

# or 

# !qstat -u $USER

## Going Further

- We highly recommend reading [dask-jobqueue documentation](http://jobqueue.dask.org/). This will get you accustomed to the jargon used, specific terms, and give you a broad overview of what to expect / not to expect from the package. **You can read the entire documentation pages including skimming through the API (Application Programming Interface) reference in less than a hour.**


- We invite you to try running the [Analysis of Gridded Ensemble Precipitation and Temperature Estimates over the Contiguous United States workflow notebook](../../workflows/gmet/gmet_ensemble.ipynb) which explores the concepts covered in this notebook in an end-to-end workflow. 